In [7]:
import CoolProp.CoolProp as CP
import pandas as pd
import numpy as np

fluids = ['Water', 'Ammonia', 'R134a']
for fluid in fluids:
    T_min = CP.PropsSI('Tmin', fluid) + 10  # Adding a small offset to Tmin
    T_max = CP.PropsSI('Tcrit', fluid)
    T_vals = np.linspace(T_min, T_max, 100)  # Number of points

    v_liquid = []
    v_vapor = []

    for T in T_vals:
        density_liquid = CP.PropsSI('D', 'T', T, 'Q', 0, fluid)  # Saturated liquid
        density_vapor = CP.PropsSI('D', 'T', T, 'Q', 1, fluid)  # Saturated vapor
        v_liquid.append(1/density_liquid)
        v_vapor.append(1/density_vapor)

    # Save to CSV
    df = pd.DataFrame({'Temperature (K)': T_vals, 'Specific Volume Liquid (m^3/kg)': v_liquid, 
                       'Specific Volume Vapor (m^3/kg)': v_vapor})
    df.to_csv(f'{fluid}_T_v.csv', index=False)


import plotly.graph_objs as go

for fluid in fluids:
    # Reading data from CSV
    df = pd.read_csv(f'{fluid}_T_v.csv')

    # Creating Plotly plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Specific Volume Liquid (m^3/kg)'], y=df['Temperature (K)'],
                             mode='lines', name='Saturated Liquid'))
    fig.add_trace(go.Scatter(x=df['Specific Volume Vapor (m^3/kg)'], y=df['Temperature (K)'],
                             mode='lines', name='Saturated Vapor'))

    # Set log scale for x-axis
    fig.update_layout(xaxis_type="log", title=f'{fluid} T-v Diagram', 
                      xaxis_title='Specific Volume (m^3/kg)', yaxis_title='Temperature (K)')

    # Save as HTML
    fig.write_html(f'{fluid}_T_v_plot.html')
